In [57]:
import pyodbc
import win32com.client as win32
import os
from datetime import datetime
import calendar
import shutil

# # String de conexão
# connection_string = (
#     f"DRIVER={{ODBC Driver 17 for SQL Server}};"
#     f"SERVER=172.17.1.35;"
#     f"DATABASE=WR_ToPy;"
#     f"UID=USR_MIS;"
#     f"PWD=MIS@321"
# )

# # Conexão com o banco
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

# # Query
# query = """
#            select 
#                 idMesRef1,
#                 idMesRef2,
#                 idMesRef3,
#                 idDiaRef1,
#                 idDiaRef2,
#                 idDiaRef3 
#             from WR_ToPy.api.tbdp
#             where convert(date,DataAtualizacao) = convert(date,getdate())
#         """
# query_valided = """
#            	select	
# 				b.Banco,
#                 a.Carteira,
#                 a.idDaily,
#                 a.validadorFinal,
#                 'Book - '+a.Carteira 'fileName'
#             from dw.glo.MapaBookPesoToNext a
# 			inner join dw.glo.dp_CarteirasToNext b on a.idDaily = b.idDaily
#             where convert(date,DataAtualizacao) = convert(date,getdate())
#         """

# # Dicionário para armazenar as linhas
# resultados_dict = {}
# resultados_dict_valided = {}

# try:
#     cursor.execute(query)
    
#     # Itera sobre os resultados e armazena no dicionário
#     for row in cursor.fetchall():
#         id_unico = f"{row.idMesRef1}_{row.idDiaRef1}"  # Pode ser um identificador único
#         resultados_dict[id_unico] = {
#             "idMesRef1": row.idMesRef1,
#             "idMesRef2": row.idMesRef2,
#             "idMesRef3": row.idMesRef3,
#             "idDiaRef1": row.idDiaRef1,
#             "idDiaRef2": row.idDiaRef2,
#             "idDiaRef3": row.idDiaRef3
#         }

#     cursor.execute(query_valided)
#     for row in cursor.fetchall():
#         id_unico_valided = f"{row.Carteira}_{row.idDaily}"  # Identificador único para o dicionário
#         resultados_dict_valided[id_unico_valided] = {
#             "Banco": row.Banco,
#             "Carteira": row.Carteira,
#             "idDaily": row.idDaily,
#             "validadorFinal": row.validadorFinal,
#             "fileName": row.fileName
#         }

#     for key, value in resultados_dict_valided.items():
#         print(f"{key}: {value}")


# except pyodbc.Error as e:
#     sqlstate = e.args[0]
#     message = e.args[1]
#     print(f"Erro SQL: {sqlstate} - Mensagem: {message}")
# finally:
#     cursor.close()
#     conn.close()






def validPath_andFiles(SubsectionName,Bank):
    #local where model file is
    path_model = rf"\\172.17.1.115\Temp\\BookToPy\\Modelos\\{SubsectionName}.xlsb"

    #validates if the file exists
    if os.path.exists(path_model):
        current_year = datetime.now().year
        year_str = str(current_year)
        path_model_to_year = rf"\\172.17.1.115\Temp\\BookToPy\\{year_str}"
        print('01.model file exists!')

        if not os.path.exists(path_model_to_year):
            os.makedirs(path_model_to_year)
            print('02.year folder created!')

            month_number = datetime.now().month

            meses_em_portugues = ['01.Janeiro','02.Fevereiro','03.Março','04.Abril','05.Maio','06.Junho',
                                  '07.Julho','08.Agosto','09.Setembro','10.Outubro','11.Novembro','12.Dezembro']
            
            mes_atual = meses_em_portugues[month_number - 1]
            path_model_to_month = rf"\\172.17.1.115\Temp\\BookToPy\\{year_str}\\{mes_atual}\\{Bank}\\{SubsectionName}\\"
            os.makedirs(path_model_to_month)
            print('03.month folder created!')

            shutil.copy(path_model,path_model_to_month)
            print('04.file copied and pasted')

            print('05. Aqui seria atualizar o arquivo e encaminhar')


        else:
            month_number = datetime.now().month

            meses_em_portugues = [
                '01.Janeiro', '02.Fevereiro', '03.Março', '04.Abril', '05.Maio', '06.Junho',
                '07.Julho', '08.Agosto', '09.Setembro', '10.Outubro', '11.Novembro', '12.Dezembro']
            
            mes_atual = meses_em_portugues[month_number - 1]
            path_model_to_month = rf"\\172.17.1.115\Temp\\BookToPy\\{year_str}\\{mes_atual}\\{Bank}\\{SubsectionName}\\"
            
            if not os.path.exists(path_model_to_month):
                os.makedirs(path_model_to_month)
                print('02. year folder exist but month s no, folder created!')

                shutil.copy(path_model,path_model_to_month)
                print('03.file copied and pasted')
                
            else:
                path_model_to_month_param = rf"\\172.17.1.115\Temp\\BookToPy\\{year_str}\\{mes_atual}\\{Bank}\\{SubsectionName}\\{SubsectionName}.xlsb"
                
                if not os.path.exists(path_model_to_month_param):
                    shutil.copy(path_model,path_model_to_month)
                    print('02.Folder month exist but file not inside! file pasted!!')
            

    else:
        print('arquivo MODELO não existe!')


validPath_andFiles('Book - CL','BV')


01.model file exists!
\\172.17.1.115\Temp\\BookToPy\\2024\\12.Dezembro\\BV\\Book - CL\\Book - CL.xlsb
02.Folder month exist but file not inside! file pasted!!


In [67]:
import win32com.client as win32
import os

def updateMacros(pathToFile):
    # Verifica se o arquivo existe
    if not os.path.exists(pathToFile):
        print(f"Arquivo não encontrado: {pathToFile}")
        return
    
    excel = None
    workbook = None  # Inicializa workbook fora do try-except
    
    try:
        # Inicia uma instância do Excel
        excel = win32.Dispatch("Excel.Application")
        excel.Visible = True  # Defina como False se não quiser mostrar o Excel

        # Abre o arquivo Excel
        workbook = excel.Workbooks.Open(Filename=pathToFile)

        # Habilita as macros (segurança baixa)
        excel.AutomationSecurity = 3  # "3" permite execução de macros sem bloqueio

        # Atualiza as células
        sheet = workbook.Sheets('Daily')

        # ID dos Meses
        sheet.Range("P1").Value = 58
        sheet.Range("Q1").Value = 59
        sheet.Range("R1").Value = 60

        # ID dos Dias
        sheet.Range("T1").Value = 18
        sheet.Range("V1").Value = 21
        sheet.Range("X1").Value = 18

        # Executa a macro
        # Verifique se o nome da macro e do módulo estão corretos
        excel.Application.Run('Atu_All.Atu_All')

        # Salva e fecha o arquivo
        workbook.Save()

    except Exception as e:
        print(f"Erro: {e}")
    
    finally:
        # Verifica se o workbook foi inicializado antes de tentar fechá-lo
        if workbook:
            workbook.Close(SaveChanges=1)  # Salva e fecha o arquivo
        if excel:
            excel.Application.Quit()  # Fecha o Excel de forma segura

# Caminho do arquivo
file_path = r"\\172.17.1.115\Temp\BookToPy\2024\12.Dezembro\BV\Book - CL\Book - CL.xlsb"

# Chama a função
updateMacros(file_path)


In [70]:
import pyodbc

# Variáveis globais
relationIds_ToVBA = {"idMesRef1": 0,
                     "idMesRef2": 0,
                     "idMesRef3": 0,
                     "idDiaRef1": 0,
                     "idDiaRef2": 0,
                     "idDiaRef3": 0}

def returnIds():
    connection_string = ( f"DRIVER={{ODBC Driver 17 for SQL Server}};"
                          f"SERVER=172.17.1.35;"
                          f"DATABASE=WR_ToPy;"
                          f"UID=USR_MIS;"
                          f"PWD=MIS@321")
    
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    query = """ select 
                    idMesRef1,
                    idMesRef2,
                    idMesRef3,
                    
                    idDiaRef1,
                    idDiaRef2,
                    idDiaRef3 

                from WR_ToPy.api.tbdp
                where convert(date,DataAtualizacao) = convert(date,getdate())
           """
    cursor.execute(query)

    for row in cursor.fetchall():
        print(row)

    cursor.close()
    conn.close()


returnIds()    


(58, 59, 60, 20, 23, 20)
